In [45]:
import pandas as pd
import numpy as np
import pprint
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import sem
from scipy.stats import linregress

# Data Load/Setup

In [46]:
# Import CSV files
#Files currently being used
emissions_df = pd.read_csv('Resources/co2_emissions_kt_by_country.csv')
pesticide_df = pd.read_csv('Resources/pesticide-use-tonnes.csv')
population_df = pd.read_csv('Resources/population.csv')

#CSV files from land folder needs cleaning
land_allotment_df = pd.read_csv('Resources/Land/agr_land_percent.csv')
total_land_df = pd.read_csv('Resources/Land/total-land.csv')

#CSV file on crop yields, needs cleaning for ease of use
crop_yields_df = pd.read_csv('Resources/crop_yields.csv')

#CSV files for GDP
agShare_df = pd.read_csv('Resources/agriculture-share-gdp.csv')
gdp_df = pd.read_csv('Resources/national-gdp-penn-world-table.csv')

## Data Cleanup/Modifying


In [47]:
# Preparing emissions, pesticide, population dataframes
emissions_df = emissions_df.rename(columns = {'country_code': 'Code','country_name': 'Country', 'year': 'Year', 'value': 'CO2 Emissions(kt)'})

pesticide_df = pesticide_df.rename(columns = {'Entity': 'Country', 
                                              'Pesticides (total) | 00001357 || Agricultural Use | 005157 || Tonnes': 'Total Pesticides(tonnes)'})

population_df = population_df.rename(columns = {'Entity': 'Country', 'Population (historical)': 'Population'})


In [48]:
print(emissions_df.head())
print(pesticide_df.head())
print(population_df.head())

  Code Country  Year  CO2 Emissions(kt)
0  ABW   Aruba  1960          11092.675
1  ABW   Aruba  1961          11576.719
2  ABW   Aruba  1962          12713.489
3  ABW   Aruba  1963          12178.107
4  ABW   Aruba  1964          11840.743
        Country Code  Year  Total Pesticides(tonnes)
0  Africa (FAO)  NaN  1990                  73958.66
1  Africa (FAO)  NaN  1991                  69956.26
2  Africa (FAO)  NaN  1992                  60260.71
3  Africa (FAO)  NaN  1993                  54301.62
4  Africa (FAO)  NaN  1994                  58819.04
       Country Code   Year  Population
0  Afghanistan  AFG -10000       14737
1  Afghanistan  AFG  -9000       20405
2  Afghanistan  AFG  -8000       28253
3  Afghanistan  AFG  -7000       39120
4  Afghanistan  AFG  -6000       54166


In [49]:
#Merging GDP based dataframes
gdp_cleaned_df = pd.merge(gdp_df, agShare_df, how = 'left', left_on = ['Entity', 'Year', 'Code'], right_on = ['Entity', 'Year', 'Code']).dropna()
gdp_cleaned_df = gdp_cleaned_df.rename(columns = {'Entity': 'Country','GDP (output, multiple price benchmarks)': 'GDP',
                                                  'Agriculture, forestry, and fishing, value added (% of GDP)': 'Ag% of GDP'})
gdp_cleaned_df['Agri. GDP'] = gdp_cleaned_df['GDP'] * gdp_cleaned_df['Ag% of GDP']
gdp_cleaned_df

,Country,Code,Year,GDP,Ag% of GDP,Agri. GDP
25,Albania,ALB,1996,15067375000,36.410862,5.486161e+11
26,Albania,ALB,1997,14072310000,31.543444,4.438891e+11
27,Albania,ALB,1998,15531704000,28.785650,4.470902e+11
28,Albania,ALB,1999,16525398000,25.918745,4.283176e+11
29,Albania,ALB,2000,16633271000,24.515411,4.077715e+11
...,...,...,...,...,...,...
10103,Zimbabwe,ZWE,2015,39798645000,8.284246,3.297018e+11
10104,Zimbabwe,ZWE,2016,40963190000,7.873986,3.225436e+11
10105,Zimbabwe,ZWE,2017,44316740000,8.340969,3.696446e+11
10106,Zimbabwe,ZWE,2018,43420897000,7.319375,3.178138e+11


In [50]:
print(land_allotment_df.columns)

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023', 'Unnamed: 68'],
      dtype='object')


In [51]:
# Modifying agricultural land allotment df
land_holder = land_allotment_df.drop(columns = ['Indicator Name', 'Indicator Code'])


df_long = pd.melt(land_holder, id_vars=['Country Name', 'Country Code'], var_name = 'Year', value_name ='% of Agricultural Land')
df_long.rename(columns={'Country Name': 'Country', 'Country Code': 'Code'}, inplace=True)
df_long_sorted_df = df_long.sort_values(by=['Country', 'Year'])
df_long_sorted_df = df_long_sorted_df.reset_index(drop = True)

indicies_to_drop = df_long_sorted_df[df_long_sorted_df['Year'] == 'Unnamed: 68'].index
df_long_sorted_df.drop(index = indicies_to_drop, inplace=True)

df_long_sorted_df['Year'] = df_long_sorted_df['Year'].astype(np.int64)
land_agri_final_df = df_long_sorted_df
land_agri_final_df

,Country,Code,Year,% of Agricultural Land
0,Afghanistan,AFG,1960,NaN
1,Afghanistan,AFG,1961,57.878356
2,Afghanistan,AFG,1962,57.955016
3,Afghanistan,AFG,1963,58.031676
4,Afghanistan,AFG,1964,58.116002
...,...,...,...,...
17284,Zimbabwe,ZWE,2019,41.876696
17285,Zimbabwe,ZWE,2020,41.876696
17286,Zimbabwe,ZWE,2021,41.876696
17287,Zimbabwe,ZWE,2022,NaN


In [52]:
#Modifying total_land_df
total_land_df
land_holder = total_land_df.drop(columns = ['Indicator Name', 'Indicator Code'])


df_long = pd.melt(land_holder, id_vars=['Country Name', 'Country Code'], var_name = 'Year', value_name ='Total Land (sq.km)')
df_long.rename(columns={'Country Name': 'Country', 'Country Code': 'Code'}, inplace=True)
df_long_sorted_df = df_long.sort_values(by=['Country', 'Year'])
df_long_sorted_df = df_long_sorted_df.reset_index(drop = True)

indicies_to_drop = df_long_sorted_df[df_long_sorted_df['Year'] == 'Unnamed: 68'].index
df_long_sorted_df.drop(index = indicies_to_drop, inplace=True)

df_long_sorted_df['Year'] = df_long_sorted_df['Year'].astype(np.int64)
total_land_final_df = df_long_sorted_df
total_land_final_df

,Country,Code,Year,Total Land (sq.km)
0,Afghanistan,AFG,1960,NaN
1,Afghanistan,AFG,1961,652230.0
2,Afghanistan,AFG,1962,652230.0
3,Afghanistan,AFG,1963,652230.0
4,Afghanistan,AFG,1964,652230.0
...,...,...,...,...
17284,Zimbabwe,ZWE,2019,386850.0
17285,Zimbabwe,ZWE,2020,386850.0
17286,Zimbabwe,ZWE,2021,386850.0
17287,Zimbabwe,ZWE,2022,NaN


In [53]:
#Merging total_land_final_df and land_agri_final_df into land_df
land_df = pd.merge(total_land_final_df, land_agri_final_df, how = 'left', left_on = ['Country', 'Year', 'Code'],
                   right_on = ['Country', 'Year', 'Code']).dropna()
land_df['Agricultural Land (sq.km)'] = land_df['Total Land (sq.km)'] * land_df['% of Agricultural Land']
land_df


,Country,Code,Year,Total Land (sq.km),% of Agricultural Land,Agricultural Land (sq.km)
1,Afghanistan,AFG,1961,652230.0,57.878356,37750000.0
2,Afghanistan,AFG,1962,652230.0,57.955016,37800000.0
3,Afghanistan,AFG,1963,652230.0,58.031676,37850000.0
4,Afghanistan,AFG,1964,652230.0,58.116002,37905000.0
5,Afghanistan,AFG,1965,652230.0,58.123668,37910000.0
...,...,...,...,...,...,...
17017,Zimbabwe,ZWE,2017,386850.0,41.876696,16200000.0
17018,Zimbabwe,ZWE,2018,386850.0,41.876696,16200000.0
17019,Zimbabwe,ZWE,2019,386850.0,41.876696,16200000.0
17020,Zimbabwe,ZWE,2020,386850.0,41.876696,16200000.0


In [54]:
# Modifying crop_yields_df so that it shows sum crop yield within each country's year
# Note: crop yields are in tonnes per hecatare

test_crop_df = crop_yields_df
all_columns = crop_yields_df.columns.tolist()
crop_columns = all_columns[2:]
crop_yields_df['Total Crop Yield (t/ha)'] = crop_yields_df[crop_columns].sum(axis=1)

crop_yields_df = test_crop_df.drop(columns=crop_columns)
crop_yields_df = crop_yields_df.rename(columns = {'country' : 'Country', 'year': 'Year'})
crop_yields_df

,Country,Year,Total Crop Yield (t/ha)
0,Afghanistan,1961,497.218199
1,Afghanistan,1962,500.500606
2,Afghanistan,1963,500.690099
3,Afghanistan,1964,508.990498
4,Afghanistan,1965,517.870401
...,...,...,...
14572,Zimbabwe,2018,780.655493
14573,Zimbabwe,2019,777.940795
14574,Zimbabwe,2020,778.620694
14575,Zimbabwe,2021,778.041794


In [55]:
#Ensuring all data has the same type of countrys/codes
print(emissions_df['Country'].nunique())
print(pesticide_df['Country'].nunique())
print(population_df['Country'].nunique())
print(gdp_cleaned_df['Country'].nunique())
print(land_df['Country'].nunique())
print(crop_yields_df['Country'].nunique())
print('---------------')
print(emissions_df['Code'].nunique())
print(pesticide_df['Code'].nunique())
print(population_df['Code'].nunique())
print(gdp_cleaned_df['Code'].nunique())
print(land_df['Code'].nunique())
# print(crop_yields_df['Code'].nunique())

256
251
271
177
258
255
---------------
255
219
250
177
258


In [56]:
#Extracting unique Codes
codes_emi = set(emissions_df['Code'].unique())
codes_pest = set(pesticide_df['Code'].unique())
codes_pop = set(population_df['Code'].unique())
codes_gdp = set(gdp_cleaned_df['Code'].unique())
codes_land = set(land_df['Code'].unique())

#Finding Common Codes
common_codes = codes_emi.intersection(codes_pest).intersection(codes_pop).intersection(codes_gdp).intersection(codes_land)

#Filter DataFrames
emissions_df_filtered = emissions_df[emissions_df['Code'].isin(common_codes)]
pesticide_df_filtered = pesticide_df[pesticide_df['Code'].isin(common_codes)]
population_df_filtered = population_df[population_df['Code'].isin(common_codes)]
gdp_cleaned_df_filtered = gdp_cleaned_df[gdp_cleaned_df['Code'].isin(common_codes)]
land_df_filtered = land_df[land_df['Code'].isin(common_codes)]

#Extracting Common Countries
country_emi = set(emissions_df_filtered['Country'].unique())
country_pest = set(pesticide_df_filtered['Country'].unique())
country_crop = set(crop_yields_df['Country'].unique())

#Finding Common Countries
common_countries = country_emi.intersection(country_pest).intersection(country_crop)

#Filter DataFrames
crop_yields_df_filtered = crop_yields_df[crop_yields_df['Country'].isin(common_countries)]


In [57]:
#Ensuring All Data has the same Country names for their codes
print(emissions_df_filtered['Country'].nunique())
print(pesticide_df_filtered['Country'].nunique())
print(population_df_filtered['Country'].nunique())
print(gdp_cleaned_df_filtered['Country'].nunique())
print(land_df_filtered['Country'].nunique())
print(crop_yields_df_filtered['Country'].nunique())
print('---------------')
print(emissions_df_filtered['Code'].nunique())
print(pesticide_df_filtered['Code'].nunique())
print(population_df_filtered['Code'].nunique())
print(gdp_cleaned_df_filtered['Code'].nunique())
print(land_df_filtered['Code'].nunique())
# print(crop_yields_df_filtered['Code'].nunique())

172
171
171
171
171
147
---------------
171
171
171
171
171


In [58]:
#Merging total_land_final_df and land_agri_final_df into land_df
agriculture_df = pd.merge(land_df_filtered, crop_yields_df_filtered, how = 'left', left_on = ['Country', 'Year'],
                   right_on = ['Country', 'Year']).dropna()


In [59]:
#Extracting all unique Years
years_emi = set(emissions_df_filtered['Year'].unique())
years_pest = set(pesticide_df_filtered['Year'].unique())
years_pop = set(population_df_filtered['Year'].unique())
years_gdp = set(gdp_cleaned_df_filtered['Year'].unique())
years_agri = set(agriculture_df['Year'].unique())

#Finding Common Codes
common_years = years_emi.intersection(years_pest).intersection(years_pop).intersection(years_gdp).intersection(years_agri)

#Filter DataFrames
emissions_final_df = emissions_df_filtered[emissions_df_filtered['Year'].isin(common_years)]
pesticide_final_df = pesticide_df_filtered[pesticide_df_filtered['Year'].isin(common_years)]
population_final_df = population_df_filtered[population_df_filtered['Year'].isin(common_years)]
gdp_final_df = gdp_cleaned_df_filtered[gdp_cleaned_df_filtered['Year'].isin(common_years)]
agriculture_final_df = agriculture_df[agriculture_df['Year'].isin(common_years)]


In [60]:
common_years

{1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019}

In [61]:
#Ensuring All Data has the same Country names for their codes
print(emissions_final_df['Country'].nunique())
print(pesticide_final_df['Country'].nunique())
print(population_final_df['Country'].nunique())
print(gdp_final_df['Country'].nunique())
print(agriculture_final_df['Country'].nunique())
print('---------------')
print(emissions_final_df['Code'].nunique())
print(pesticide_final_df['Code'].nunique())
print(population_final_df['Code'].nunique())
print(gdp_final_df['Code'].nunique())
print(agriculture_final_df['Code'].nunique())
print('---------------')
print(emissions_final_df['Year'].nunique())
print(pesticide_final_df['Year'].nunique())
print(population_final_df['Year'].nunique())
print(gdp_final_df['Year'].nunique())
print(agriculture_final_df['Year'].nunique())

171
171
171
171
145
---------------
171
171
171
171
145
---------------
30
30
30
30
30


In [62]:
#Extracting Common Countries
country_emi= set(emissions_final_df['Country'].unique())
country_pest = set(pesticide_final_df['Country'].unique())
country_pop = set(population_final_df['Country'].unique())
country_gdp = set(gdp_final_df['Country'].unique())
country_agri = set(agriculture_final_df['Country'].unique())

#Finding Common Countries
common_countries = country_emi.intersection(country_emi).intersection(country_pest).intersection(country_pop).intersection(country_gdp).intersection(country_agri)

#Filter DataFrames
emi_final_df = emissions_final_df[emissions_final_df['Country'].isin(common_countries)]
pest_final_df = pesticide_final_df[pesticide_final_df['Country'].isin(common_countries)]
pop_final_df = population_final_df[population_final_df['Country'].isin(common_countries)]
gdpA_final_df = gdp_final_df[gdp_final_df['Country'].isin(common_countries)]
agri_final_df = agriculture_final_df[agriculture_final_df['Country'].isin(common_countries)]

In [63]:
#Ensuring All Data has the same Country names for their codes
print(emi_final_df['Country'].nunique())
print(pest_final_df['Country'].nunique())
print(pop_final_df['Country'].nunique())
print(gdpA_final_df['Country'].nunique())
print(agri_final_df['Country'].nunique())
print('---------------')
print(emi_final_df['Code'].nunique())
print(pest_final_df['Code'].nunique())
print(pop_final_df['Code'].nunique())
print(gdpA_final_df['Code'].nunique())
print(agri_final_df['Code'].nunique())
print('---------------')
print(emi_final_df['Year'].nunique())
print(pest_final_df['Year'].nunique())
print(pop_final_df['Year'].nunique())
print(gdpA_final_df['Year'].nunique())
print(agri_final_df['Year'].nunique())

145
145
145
145
145
---------------
145
145
145
145
145
---------------
30
30
30
30
30


# Importing Cleaned Data into SQLlite database


In [64]:
#Importing sqlite library
import sqlite3
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, MetaData

In [65]:
print(sqlalchemy.__version__)

2.0.30


In [66]:
#Conn to new SQLite databse, will create new one
conn = sqlite3.connect('project3_database.sqlite')

In [67]:
#Importing Cleaned dataframes into SQLlite database
emi_final_df.to_sql('emissions', conn, if_exists='replace', index=False)
pest_final_df.to_sql('pesticides', conn, if_exists='replace', index=False)
pop_final_df.to_sql('population', conn, if_exists = 'replace', index=False)
gdpA_final_df.to_sql('gdp', conn, if_exists = 'replace', index=False)
agri_final_df.to_sql('agriculture', conn, if_exists = 'replace', index=False)

4238

In [68]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
print(tables)

          name
0    emissions
1   pesticides
2   population
3          gdp
4  agriculture


In [69]:
df_check = pd.read_sql('SELECT * FROM pesticides LIMIT 5;', conn)
print(df_check)

   Country Code  Year  Total Pesticides(tonnes)
0  Albania  ALB  1990                     121.0
1  Albania  ALB  1991                     121.0
2  Albania  ALB  1992                     121.0
3  Albania  ALB  1993                     121.0
4  Albania  ALB  1994                     201.0


In [70]:
conn.close()

In [71]:
#Verifying database works with SQLalchemy
# Create an engine to connect to the SQLite database
engine = create_engine("sqlite:///project3_database.sqlite")

# Create a MetaData object
metadata = MetaData()

# Reflect the tables
metadata.reflect(bind=engine)

# Create a base class for automapping
Base = automap_base(metadata=metadata)

# Prepare the base to map the tables
Base.prepare(engine)

In [74]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker

# Create an engine to connect to the SQLite database
engine = create_engine("sqlite:///project3_database.sqlite")

# Create a MetaData object
metadata = MetaData()

# Reflect the tables
metadata.reflect(bind=engine)

# Create a base class for automapping
Base = automap_base(metadata=metadata)

# Prepare the base to map the tables
Base.prepare(engine)

# View all of the classes (tables) that automap found
print("Tables found:", Base.classes.keys())

# Access tables by name if they are found
if 'emissions' in Base.classes:
    Emissions = Base.classes.emissions
    print("Emissions table is available.")
else:
    print("Emissions table not found.")

# Example: Query the emissions table
Session = sessionmaker(bind=engine)
session = Session()

# Query the emissions table if it exists
if 'emissions' in Base.classes:
    query_emissions = session.query(Base.classes.emissions).first()
    print(query_emissions)

# Close the session
session.close()


Tables found: []
Emissions table not found.


In [75]:
print("Tables in metadata:", metadata.tables.keys())

Tables in metadata: dict_keys(['agriculture', 'emissions', 'gdp', 'pesticides', 'population'])


In [76]:
import sqlite3

conn = sqlite3.connect('project3_database.sqlite')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:", [table[0] for table in tables])

conn.close()


Tables in database: ['emissions', 'pesticides', 'population', 'gdp', 'agriculture']


In [77]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.automap import automap_base

# Create engine and reflect tables
engine = create_engine("sqlite:///project3_database.sqlite")

# Create a MetaData object
metadata = MetaData()

# Reflect the tables
metadata.reflect(bind=engine)

# Create a base class for automapping
Base = automap_base(metadata=metadata)

# Prepare the base to map the tables
Base.prepare(engine)

# Print the keys (table names) that automap found
print("Tables found:", Base.classes.keys())


Tables found: []


In [78]:
print("Tables in metadata:", metadata.tables.keys())

Tables in metadata: dict_keys(['agriculture', 'emissions', 'gdp', 'pesticides', 'population'])
